# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherDf = pd.read_csv("WeatherPy.csv")
weatherDf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        572 non-null    object 
 1   Lat         572 non-null    float64
 2   Lng         572 non-null    float64
 3   Max Temp    572 non-null    float64
 4   Humidity    572 non-null    int64  
 5   Cloudiness  572 non-null    int64  
 6   Wind Speed  572 non-null    float64
 7   Country     565 non-null    object 
 8   Date        572 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 40.3+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
latLong = weatherDf[['Lat','Lng']]
Humidity = weatherDf['Humidity']
print("test")


test


In [4]:
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
heat_layer = gmaps.heatmap_layer(latLong, weights=Humidity,dissipating=False, max_intensity=100,point_radius=1.5)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
idealWeatherDf = pd.DataFrame()

ideal_temp = (weatherDf['Max Temp'] < 80) & (weatherDf['Max Temp'] >70)
ideal_windSpeed = weatherDf['Wind Speed'] < 15
ideal_cloudiness = weatherDf['Cloudiness'] == 0
ideal_conditions = ideal_temp & ideal_windSpeed & ideal_cloudiness

ideal_locations = weatherDf[ideal_conditions].dropna().reset_index()
ideal_locations.head(10)
print(len(ideal_locations))

21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
from pprint import pprint

hotel_df = ideal_locations
hotel_df['Hotel Name'] = ""

jsonList = []
for city, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["Lat"]
    lng = row['Lng']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "type": "hotel",
        "key": g_key}
    
    response = requests.get(base_url, params=params)
    response =response.json()
   
    jsonList.append(response['results'][1]['name'])

hotel_df['Hotel Name']=jsonList
hotel_df.head()


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,46,carutapera,-1.20,-46.02,77.810,80,0,3.72,BR,2020-08-20 19:11:27,CARU BALADAS
1,79,koroni,36.80,21.95,78.008,88,0,2.10,GR,2020-08-20 19:12:36,Apartments Xenios Zeus 1
2,116,waddan,29.16,16.14,74.480,66,0,3.93,LY,2020-08-20 19:12:54,Alternative (Authorized agent for the company'...
3,147,soma,39.19,27.61,71.600,72,0,0.50,TR,2020-08-20 19:13:03,Hotel Labella
4,172,canakkale,40.08,26.83,71.600,56,0,2.10,TR,2020-08-20 19:13:09,Övezdere Köy Meydanı


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# Add marker layer ontop of heat map

hotelMap = gmaps.figure()
tags = gmaps.marker_layer(locations)
hotelMap.add_layer(tags)

hotelMap



# Display figure


Figure(layout=FigureLayout(height='420px'))